In [1]:
import chainer
import chainer.links as L
import chainer.functions as F
import chainer.optimizers as opt
from chainer.dataset import concat_examples
from chainer.backends import cuda

import numpy as np
from functools import partial

#from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm

import models.VGG

# DataLoader(CIFAR10)

In [2]:
batch_size = 100
def transform(in_data, std, mean):
    img, label = in_data
    img -= np.asarray(mean)[:, np.newaxis, np.newaxis]
    img /= np.asarray(std)[:, np.newaxis, np.newaxis]
    return img, label
_, testset = chainer.datasets.cifar.get_cifar10(withlabel=True, ndim=3, scale=1.0)

testset_trans = chainer.datasets.TransformDataset(testset, partial(transform, std=[0.5, 0.5, 0.5], mean=[0.5, 0.5, 0.5]))
test_loader = chainer.iterators.MultiprocessIterator(testset_trans, batch_size=batch_size, shuffle = False, n_processes=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Model

In [3]:
#model = L.VGG16Layers()
#model.fc6 = L.Linear(512, 4096)
#model.fc8 = L.Linear(4096, len(classes))
#model.available_layers

#model = L.Classifier(models.VGG.VGG(len(classes)))
model = models.VGG.VGG(len(classes))
chainer.serializers.load_npz("params/model.npz", model)

gpu_id = 0  # Set to -1 if you use CPU
if gpu_id >= 0:
    model.to_gpu(gpu_id)

# Inference function

In [4]:
def inference_loop(model, loader, num_steps, gpu_id):
    with chainer.using_config('train', False):
        bar = tqdm(total=num_steps, leave=False)
        total_acc, total_num = 0, 0
        while True:
            feed = loader.next()
            # Prepare data
            inputs, labels = concat_examples(feed, gpu_id)
            # Foward
            #outputs = model.forward(inputs, layers=['fc8'])['fc8']
            outputs = model(inputs)
            # Update bar
            ## Accuracy
            accuracy = F.accuracy(outputs, labels)
            accuracy.to_cpu()
            ## Calcurate Score
            total_acc += accuracy.data
            total_num += len(labels)

            #print(total_loss / total_num, total_acc / total_num * 100)
            bar.set_description("Accuracy: {:.2f}".format(total_acc / total_num * 100), refresh=True)
            bar.update()
            
            if loader.is_new_epoch:
                bar.close()
                break
                
    return total_num, total_acc / total_num * 100

In [5]:
data_num, acc = inference_loop(model, test_loader, len(testset_trans)/batch_size, gpu_id)
print('Data num: {}, Test Accuracy: {:.2f}'.format(data_num, acc))

Data num: 10000, Test Accuracy: 0.66
